# Random Forest modelling (SimFin local) — paper-faithful walk-forward backtest (improved)

This notebook refactors `RF_Modelling_SimFin_Local_v3.ipynb` to better reproduce the methodology in **Morgan Wynne (2023)**:

- **Investment date:** last month-end of each year (typically **31/12/YYYY**).
- **No look-ahead:** for an investment date `D`, train uses data **up to 31/12/(Y-1)** (one year before `D`), so training targets (1-year forward returns) cannot peek beyond `D`.
- **Walk-forward evaluation:** repeat annually across decades (first test year defaults to **min_year + 11**, matching “10 years of history”).
- **Time-respecting hyperparameter tuning:** optional **purged, date-based CV** (no random shuffling).
- **Missing data handling:** offers a paper-faithful **drop strategy** *and* safer imputation alternatives (so you can see the trade-offs explicitly).

> Tip: keep `MISSING_STRATEGY="paper_drop"` and `RF_PARAMS_PAPER` for the closest reproduction.

In [ ]:
# Core deps
import os
import json
import math
import warnings
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional, Tuple, Iterable

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid, ParameterSampler

from scipy.stats import ttest_ind, spearmanr

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)

## 1) Configuration

Keep all “magic numbers” here so the experiment is fully reproducible.

In [ ]:
@dataclass(frozen=True)
class Config:
    # ---- Data ----
    PANEL_PATH: str = "data/simfin_panel.csv"
    DATE_COL: str = "public_date"
    ID_COL: str = "TICKER"
    TARGET_COL: str = "1yr_return"

    # ---- Universe controls ----
    CAP_COL: str = "cap"                  # optional (ignored if missing)
    ALLOW_CAPS: Optional[List[str]] = None  # e.g. ["Mid Cap","Large Cap","Mega Cap"]

    # ---- Backtest setup (paper-faithful) ----
    # First investment date defaults to min_year + 11 (10y history, then first invest at year-end).
    FIRST_TEST_YEAR: Optional[int] = None
    LAST_TEST_YEAR: Optional[int] = None  # inclusive; default auto based on data coverage

    # For investment date D in year Y:
    # train_end = Dec 31 of (Y-1)  (paper: “one year prior” to avoid target peeking)
    PURGE_MONTHS: int = 12

    # ---- Feature selection ----
    FEATURE_MODE: str = "auto_numeric"  # "auto_numeric" | "manual"
    MANUAL_FEATURES: Optional[List[str]] = None

    # ---- Missing data handling ----
    # "paper_drop": drop features with too much missingness (based on training only),
    #               then drop any rows with missing among remaining features.
    # "median_impute": median impute (fit on training only), optionally add missing indicators
    # "ffill_then_median": per-ticker forward-fill then median impute
    MISSING_STRATEGY: str = "paper_drop"
    ADD_MISSING_INDICATORS: bool = True  # for imputation strategies

    # Paper drops features with >120k missing out of ~2.4M rows (~5%).
    # We express this as a rate, applied per *training window*.
    MAX_FEATURE_MISSING_RATE: float = 0.05

    # ---- Model params ----
    # Closest to the thesis:
    # - max_features = sqrt(#features)
    # - bootstrap sample size per tree = 10%
    RF_PARAMS_PAPER: Dict = None

    # ---- Portfolio sizes ----
    PORTFOLIO_SIZES: Tuple[int, ...] = (30, 50, 100, 200)

    # ---- Outputs ----
    OUT_DIR: str = "outputs/rf_simfin_local_v4"

def default_rf_params_paper():
    return dict(
        n_estimators=350,
        max_depth=13,
        max_features="sqrt",
        bootstrap=True,
        max_samples=0.10,     # 10% bootstrap sample per tree (paper)
        n_jobs=-1,
        random_state=42,
    )

CFG = Config(RF_PARAMS_PAPER=default_rf_params_paper())

os.makedirs(CFG.OUT_DIR, exist_ok=True)
print("OUT_DIR:", CFG.OUT_DIR)

## 2) Load data (no global target filtering)

Important: we **do not** drop missing targets globally, to avoid changing the sample composition for earlier years.
Targets are filtered **inside each walk-forward iteration**.

In [ ]:
def load_panel(cfg: Config) -> pd.DataFrame:
    df = pd.read_csv(cfg.PANEL_PATH)
    df[cfg.DATE_COL] = pd.to_datetime(df[cfg.DATE_COL], errors="coerce")

    # Basic hygiene
    df = df.dropna(subset=[cfg.DATE_COL]).copy()
    df = df.sort_values([cfg.DATE_COL, cfg.ID_COL]).reset_index(drop=True)

    # Ensure target is numeric but do NOT drop NAs here
    df[cfg.TARGET_COL] = pd.to_numeric(df[cfg.TARGET_COL], errors="coerce")
    df[cfg.TARGET_COL] = df[cfg.TARGET_COL].replace([np.inf, -np.inf], np.nan)

    return df

df = load_panel(CFG)
print("Rows:", len(df))
print("Date range:", df[CFG.DATE_COL].min(), "->", df[CFG.DATE_COL].max())
print("Columns:", len(df.columns))

## 3) Feature set

To match the thesis most closely, you typically want all available **numeric** fundamental/macro features, excluding identifiers and the target.
You can switch to `FEATURE_MODE="manual"` if you have a fixed “paper feature list”.

In [ ]:
NON_FEATURE_COLS = {CFG.DATE_COL, CFG.ID_COL, CFG.TARGET_COL, "gvkey", "public_date", "Fiscal Year", "Fiscal Period", "Publish Date"}

def infer_features(df: pd.DataFrame, cfg: Config) -> List[str]:
    if cfg.FEATURE_MODE == "manual":
        if not cfg.MANUAL_FEATURES:
            raise ValueError("FEATURE_MODE='manual' but MANUAL_FEATURES is empty.")
        feats = [c for c in cfg.MANUAL_FEATURES if c in df.columns]
        missing = set(cfg.MANUAL_FEATURES) - set(feats)
        if missing:
            print("[WARN] Missing manual features:", sorted(missing)[:20], "...")
        return feats

    # auto_numeric
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    feats = [c for c in numeric_cols if c not in NON_FEATURE_COLS]
    return feats

ALL_FEATURES = infer_features(df, CFG)
print("Inferred feature count:", len(ALL_FEATURES))
print("Example features:", ALL_FEATURES[:20])

## 4) Missingness diagnostics (global)

This does **not** drive feature selection (that happens per training window), but it helps you understand where missingness comes from.

In [ ]:
def missing_report(df: pd.DataFrame, features: List[str], top_n: int = 30) -> pd.DataFrame:
    miss = df[features].isna().mean().sort_values(ascending=False)
    rep = pd.DataFrame({"missing_rate": miss, "missing_count": (miss * len(df)).round().astype(int)})
    return rep.head(top_n)

miss_top = missing_report(df, ALL_FEATURES, top_n=30)
display(miss_top)

## 5) Date utilities and paper-faithful investment schedule

In [ ]:
def last_month_end_in_year(df: pd.DataFrame, date_col: str, year: int) -> Optional[pd.Timestamp]:
    s = df.loc[df[date_col].dt.year == year, date_col]
    if len(s) == 0:
        return None
    return pd.to_datetime(s.max())

def year_end(year: int) -> pd.Timestamp:
    return pd.Timestamp(year=year, month=12, day=31)

def auto_test_year_range(df: pd.DataFrame, cfg: Config) -> Tuple[int, int]:
    min_year = int(df[cfg.DATE_COL].dt.year.min())
    max_year = int(df[cfg.DATE_COL].dt.year.max())

    first = cfg.FIRST_TEST_YEAR if cfg.FIRST_TEST_YEAR is not None else (min_year + 11)
    # Need at least one more year after investment date to have 1yr_return observed in test set
    last = cfg.LAST_TEST_YEAR if cfg.LAST_TEST_YEAR is not None else (max_year - 1)
    return first, last

FIRST_Y, LAST_Y = auto_test_year_range(df, CFG)
print("Test years:", FIRST_Y, "->", LAST_Y)

## 6) Missing-data handling per window

Two modes:
- **paper_drop** (closest to thesis):  
  1) drop features whose **training-window** missing rate exceeds `MAX_FEATURE_MISSING_RATE`,  
  2) drop rows with any missing in remaining features (train and test separately),  
  3) fit RF on complete-case.

- **imputation modes**: safer for “real” trading pipelines, and useful to quantify selection bias.

In [ ]:
def apply_universe_filter(frame: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    if cfg.ALLOW_CAPS is None:
        return frame
    if cfg.CAP_COL not in frame.columns:
        print("[WARN] CAP_COL not present; ignoring ALLOW_CAPS.")
        return frame
    return frame[frame[cfg.CAP_COL].isin(cfg.ALLOW_CAPS)].copy()

def select_features_paper_drop(train_df: pd.DataFrame, features: List[str], cfg: Config) -> List[str]:
    miss_rate = train_df[features].isna().mean()
    keep = miss_rate[miss_rate <= cfg.MAX_FEATURE_MISSING_RATE].index.tolist()
    # Also drop features that are entirely missing (can happen in short windows)
    keep = [c for c in keep if train_df[c].notna().any()]
    return keep

def prep_Xy(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    features: List[str],
    cfg: Config,
):
    # Filter to rows where target is known in each set (NO global filtering)
    train_df = train_df.dropna(subset=[cfg.TARGET_COL]).copy()
    test_df_eval = test_df.dropna(subset=[cfg.TARGET_COL]).copy()

    if cfg.MISSING_STRATEGY == "paper_drop":
        keep_feats = select_features_paper_drop(train_df, features, cfg)

        tr = train_df[keep_feats + [cfg.TARGET_COL]].dropna().copy()
        te = test_df_eval[keep_feats + [cfg.TARGET_COL]].dropna().copy()

        Xtr, ytr = tr[keep_feats], tr[cfg.TARGET_COL]
        Xte, yte = te[keep_feats], te[cfg.TARGET_COL]

        prep = {"features": keep_feats, "strategy": "paper_drop", "rows_train": len(tr), "rows_test": len(te)}
        return Xtr, ytr, Xte, yte, prep

    # For imputation strategies, optionally forward fill within ticker first
    tr = train_df.copy()
    te = test_df_eval.copy()

    if cfg.MISSING_STRATEGY == "ffill_then_median":
        if cfg.ID_COL in tr.columns:
            tr = tr.sort_values([cfg.ID_COL, cfg.DATE_COL]).copy()
            te = te.sort_values([cfg.ID_COL, cfg.DATE_COL]).copy()
            tr[features] = tr.groupby(cfg.ID_COL)[features].ffill()
            te[features] = te.groupby(cfg.ID_COL)[features].ffill()

    # Fit imputer only on training
    imputer = SimpleImputer(strategy="median")
    Xtr_raw = tr[features]
    Xte_raw = te[features]

    Xtr_imp = pd.DataFrame(imputer.fit_transform(Xtr_raw), columns=features, index=tr.index)
    Xte_imp = pd.DataFrame(imputer.transform(Xte_raw), columns=features, index=te.index)

    # Add missing indicators if requested
    if cfg.ADD_MISSING_INDICATORS:
        tr_miss = Xtr_raw.isna().astype(np.int8).add_prefix("miss__")
        te_miss = Xte_raw.isna().astype(np.int8).add_prefix("miss__")
        Xtr_imp = pd.concat([Xtr_imp, tr_miss], axis=1)
        Xte_imp = pd.concat([Xte_imp, te_miss], axis=1)

    ytr = tr[cfg.TARGET_COL].astype(float)
    yte = te[cfg.TARGET_COL].astype(float)

    prep = {"features": list(Xtr_imp.columns), "strategy": cfg.MISSING_STRATEGY, "rows_train": len(tr), "rows_test": len(te)}
    return Xtr_imp, ytr, Xte_imp, yte, prep

## 7) Time-respecting hyperparameter tuning (optional)

The thesis uses a 10% bootstrap sample and 5-fold CV. The key fix here is: **do not shuffle time**.

We implement a **purged, date-based CV**:
- split by unique dates,
- ensure the training fold ends at least `PURGE_MONTHS` before the validation fold (to avoid overlapping 1y-forward targets).

In [ ]:
def purged_date_splits(
    df_train: pd.DataFrame,
    cfg: Config,
    n_splits: int = 5,
) -> List[Tuple[np.ndarray, np.ndarray]]:
    # split on unique dates, not rows
    dates = pd.to_datetime(df_train[cfg.DATE_COL].unique())
    dates = np.array(sorted(dates))

    if len(dates) < (n_splits + 2):
        return []

    # Split dates into consecutive blocks for validation
    folds = np.array_split(dates, n_splits + 1)  # last block can be unused buffer
    splits = []
    purge = pd.DateOffset(months=cfg.PURGE_MONTHS)

    for i in range(n_splits):
        val_dates = folds[i+1]  # skip first block so each split has some history
        if len(val_dates) == 0:
            continue
        val_start = val_dates.min()

        # Purge: training must end at least 12 months before val_start
        train_end = val_start - purge
        train_mask = df_train[cfg.DATE_COL] <= train_end
        val_mask = df_train[cfg.DATE_COL].isin(val_dates)

        tr_idx = df_train.index[train_mask].to_numpy()
        va_idx = df_train.index[val_mask].to_numpy()
        if len(tr_idx) and len(va_idx):
            splits.append((tr_idx, va_idx))

    return splits

def tune_rf(
    df_train_window: pd.DataFrame,
    features: List[str],
    cfg: Config,
    param_grid: Dict,
    n_iter: int = 25,
    sample_frac: float = 0.10,
    random_state: int = 42,
):
    # Bootstrap sample (paper uses 10% for grid search feasibility)
    df_samp = df_train_window.sample(frac=min(sample_frac, 1.0), random_state=random_state)

    # Prepare data using same missing strategy (paper_drop recommended for closest replication)
    Xtr, ytr, _, _, prep = prep_Xy(df_samp, df_samp, features, cfg)
    if len(Xtr) < 1000:
        print("[WARN] Very small tuning sample after prep:", len(Xtr))

    splits = purged_date_splits(df_samp.dropna(subset=[cfg.TARGET_COL]), cfg, n_splits=5)
    if not splits:
        print("[WARN] Not enough unique dates for purged CV; skipping tuning.")
        return cfg.RF_PARAMS_PAPER

    rng = np.random.RandomState(random_state)
    candidates = list(ParameterSampler(param_grid, n_iter=n_iter, random_state=rng))

    best = None
    best_mse = np.inf

    # Map df index to positions in Xtr
    idx_to_pos = {idx: pos for pos, idx in enumerate(Xtr.index.to_numpy())}

    for p in candidates:
        model = RandomForestRegressor(**{**cfg.RF_PARAMS_PAPER, **p})

        fold_mses = []
        for tr_idx, va_idx in splits:
            tr_pos = [idx_to_pos[i] for i in tr_idx if i in idx_to_pos]
            va_pos = [idx_to_pos[i] for i in va_idx if i in idx_to_pos]
            if len(tr_pos) == 0 or len(va_pos) == 0:
                continue

            model.fit(Xtr.iloc[tr_pos], ytr.iloc[tr_pos])
            pred = model.predict(Xtr.iloc[va_pos])
            fold_mses.append(mean_squared_error(ytr.iloc[va_pos], pred))

        if not fold_mses:
            continue
        mse = float(np.mean(fold_mses))
        if mse < best_mse:
            best_mse = mse
            best = p

    if best is None:
        return cfg.RF_PARAMS_PAPER

    tuned = {**cfg.RF_PARAMS_PAPER, **best}
    print("Best CV MSE:", best_mse, "Best params delta:", best)
    return tuned

# Example tuning grid (expanded vs v3; still practical)
PARAM_GRID = {
    "max_depth": list(range(3, 16)),
    "n_estimators": list(range(100, 501, 50)),
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    "max_features": ["sqrt", 0.33, 0.5],
}

## 8) Walk-forward backtest (paper-faithful)

For each test year Y:

- `investment_date` = last month-end in Y (ideally 31/12/Y)
- `train_end` = 31/12/(Y-1)
- training data = all rows with `date <= train_end`
- test data = all rows with `date == investment_date`

We fit once per year and rank stocks by predicted 1-year return.

In [ ]:
def top_n_mean_return(y_true: np.ndarray, y_pred: np.ndarray, n: int) -> float:
    if len(y_pred) == 0:
        return float("nan")
    n = min(n, len(y_pred))
    idx = np.argsort(y_pred)[-n:]
    return float(np.mean(y_true[idx]))

def run_walk_forward_rf(
    df: pd.DataFrame,
    features: List[str],
    cfg: Config,
    rf_params: Optional[Dict] = None,
    do_tune: bool = False,
) -> pd.DataFrame:
    rf_params = rf_params or cfg.RF_PARAMS_PAPER

    df = df.copy()
    df = apply_universe_filter(df, cfg)

    first_y, last_y = auto_test_year_range(df, cfg)

    rows = []
    yearly_preds = []

    for year in range(first_y, last_y + 1):
        invest_date = last_month_end_in_year(df, cfg.DATE_COL, year)
        if invest_date is None:
            rows.append({
                "year": year,
                "investment_date": pd.NaT,
                "train_end": year_end(year-1),
                "status": "SKIP_NO_DATE",
                "features": None,
                "strategy": None,
                "rows_train": None,
                "rows_test": None,
            })
            continue

        train_end = year_end(year - 1)

        df_train = df[df[cfg.DATE_COL] <= train_end].copy()
        df_test = df[df[cfg.DATE_COL] == invest_date].copy()

        if len(df_train) == 0 or len(df_test) == 0:
            rows.append({
                "year": year,
                "investment_date": invest_date,
                "train_end": train_end,
                "status": "SKIP_EMPTY_WINDOW",
                "features": None,
                "strategy": None,
                "rows_train": int(len(df_train)),
                "rows_test": int(len(df_test)),
            })
            continue

        # Optional time-respecting tuning on this window
        params_here = rf_params
        if do_tune:
            params_here = tune_rf(df_train, features, cfg, PARAM_GRID, n_iter=40, sample_frac=0.10)

        Xtr, ytr, Xte, yte, prep = prep_Xy(df_train, df_test, features, cfg)
        if len(Xtr) < 1000 or len(Xte) < 50:
            rows.append({
                "year": year, "investment_date": invest_date, "train_end": train_end,
                "status": "SKIP_SMALL",
                **prep
            })
            continue

        model = RandomForestRegressor(**params_here)
        model.fit(Xtr, ytr)
        pred = model.predict(Xte)

        # Metrics aligned with portfolio use-case
        mse = float(mean_squared_error(yte, pred))
        rho, rho_p = spearmanr(pred, yte)  # ranking agreement

        out = {
            "year": year,
            "investment_date": invest_date,
            "train_end": train_end,
            "status": "OK",
            "mse": mse,
            "spearman_r": float(rho),
            "spearman_p": float(rho_p),
            "num_train": int(len(Xtr)),
            "num_test_eval": int(len(Xte)),
            "rows_train": int(prep.get("rows_train", len(Xtr))),
            "rows_test": int(prep.get("rows_test", len(Xte))),
            "strategy": prep.get("strategy"),
            "features": prep.get("features"),
            "num_features_used": int(len(prep["features"])),
            "missing_strategy": prep["strategy"],
        }

        # Portfolio returns (equal-weight top N)
        for n in cfg.PORTFOLIO_SIZES:
            out[f"ret_top_{n}"] = top_n_mean_return(np.asarray(yte), pred, n)

        rows.append(out)

        pred_frame = df_test.loc[Xte.index, [cfg.ID_COL, cfg.DATE_COL]].copy()
        pred_frame["y_true"] = yte.values
        pred_frame["y_pred"] = pred
        pred_frame["rank_pred_desc"] = (-pred_frame["y_pred"]).rank(method="first")
        pred_frame["test_year"] = year
        yearly_preds.append(pred_frame)

        print(f"[OK] {year} invest={invest_date.date()} train_end={train_end.date()} "
              f"train={len(Xtr)} test_eval={len(Xte)} feats={len(prep['features'])} mse={mse:.4f} rho={rho:.3f}")

    results = pd.DataFrame(rows)
    if results.empty or ("year" not in results.columns):
        print("[WARN] No results produced. Check date alignment, universe filters, and target availability.")
        # Return an empty, schema-stable DataFrame
        results = pd.DataFrame(columns=[
            "year","investment_date","train_end","status","mse","spearman_r","spearman_p",
            "num_train","num_test_eval","num_features_used","prep_mode","impute_mode",
            "num_features_in","num_features_dropped_missing","num_rows_dropped_missing"
        ])
        return results
    results = results.sort_values("year").reset_index(drop=True)

    # Persist
    results.to_csv(os.path.join(cfg.OUT_DIR, "rf_walkforward_summary.csv"), index=False)
    if yearly_preds:
        pd.concat(yearly_preds, ignore_index=True).to_csv(os.path.join(cfg.OUT_DIR, "rf_yearly_predictions.csv"), index=False)

    # Store config snapshot
    with open(os.path.join(cfg.OUT_DIR, "run_config.json"), "w") as f:
        json.dump(asdict(cfg), f, default=str, indent=2)

    return results

summary = run_walk_forward_rf(df, ALL_FEATURES, CFG, rf_params=CFG.RF_PARAMS_PAPER, do_tune=False)
display(summary.tail(10))
print("Wrote:", os.path.join(CFG.OUT_DIR, "rf_walkforward_summary.csv"))

## 9) Performance aggregation (paper-style)

We compute:
- mean annual return for each portfolio size,
- volatility (std),
- excess return vs a benchmark (optional),
- **SIR** (mean excess return / std excess return),
- 1-tailed Welch’s t-test vs benchmark (if provided).

If you have a benchmark series, add it here. Otherwise, you can treat the market average return as a rough proxy.

In [ ]:
def aggregate_portfolio_metrics(summary: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    ok = summary[summary["status"] == "OK"].copy()
    if ok.empty:
        return pd.DataFrame()

    out_rows = []
    for n in cfg.PORTFOLIO_SIZES:
        r = ok[f"ret_top_{n}"].astype(float).dropna()
        out_rows.append({
            "portfolio_n": n,
            "years": int(len(r)),
            "mean_return": float(r.mean()),
            "std_return": float(r.std(ddof=1)) if len(r) > 1 else float("nan"),
        })
    return pd.DataFrame(out_rows)

metrics = aggregate_portfolio_metrics(summary, CFG)
display(metrics)

## 10) Cumulative performance plots (optional)

In [ ]:
import matplotlib.pyplot as plt

def plot_cumulative(summary: pd.DataFrame, cfg: Config, n: int):
    ok = summary[summary["status"] == "OK"].sort_values("year").copy()
    if ok.empty:
        print("No OK rows.")
        return
    rets = ok[f"ret_top_{n}"].astype(float).fillna(0.0).values
    years = ok["year"].values
    cum = np.cumprod(1.0 + rets)

    plt.figure()
    plt.plot(years, cum)
    plt.xlabel("Year")
    plt.ylabel("Cumulative growth (start=1.0)")
    plt.title(f"RF top-{n} portfolio cumulative growth")
    plt.show()

for n in CFG.PORTFOLIO_SIZES:
    plot_cumulative(summary, CFG, n)

## 11) Feature importance (averaged across test years)

This reproduces the “ordered feature importances” analysis from the thesis.
For imputation strategies with missing indicators, importances will include `miss__*` features.

In [ ]:
def compute_feature_importance(
    df: pd.DataFrame,
    features: List[str],
    cfg: Config,
    rf_params: Dict,
) -> pd.DataFrame:
    df = apply_universe_filter(df, cfg)
    first_y, last_y = auto_test_year_range(df, cfg)

    imp_rows = []
    for year in range(first_y, last_y + 1):
        invest_date = last_month_end_in_year(df, cfg.DATE_COL, year)
        if invest_date is None:
            continue
        train_end = year_end(year - 1)
        df_train = df[df[cfg.DATE_COL] <= train_end].copy()
        df_test = df[df[cfg.DATE_COL] == invest_date].copy()

        Xtr, ytr, Xte, yte, prep = prep_Xy(df_train, df_test, features, cfg)
        if len(Xtr) < 1000:
            continue

        model = RandomForestRegressor(**rf_params)
        model.fit(Xtr, ytr)

        imps = pd.Series(model.feature_importances_, index=prep["features"])
        imps = imps.sort_values(ascending=False).head(50)
        row = {"year": year, **{k: float(v) for k, v in imps.items()}}
        imp_rows.append(row)

    if not imp_rows:
        return pd.DataFrame()

    imp_df = pd.DataFrame(imp_rows).fillna(0.0)
    mean_imp = imp_df.drop(columns=["year"]).mean().sort_values(ascending=False)
    out = pd.DataFrame({"feature": mean_imp.index, "mean_importance": mean_imp.values})
    return out

feat_imp = compute_feature_importance(df, ALL_FEATURES, CFG, CFG.RF_PARAMS_PAPER)
display(feat_imp.head(30))
feat_imp.to_csv(os.path.join(CFG.OUT_DIR, "rf_feature_importance_mean.csv"), index=False)

## 12) Reproducing the paper’s “Mid Cap and larger” restriction

Set `ALLOW_CAPS` and rerun. (This requires the panel to include a `cap` column.)

In [ ]:
# Example: Mid Cap restriction (paper compares unrestricted vs Mid Cap+)
CFG_MID = Config(**{**asdict(CFG), "ALLOW_CAPS": ["Mid Cap", "Large Cap", "Mega Cap"], "OUT_DIR": os.path.join(CFG.OUT_DIR, "midcap_plus")})
os.makedirs(CFG_MID.OUT_DIR, exist_ok=True)

summary_mid = run_walk_forward_rf(df, ALL_FEATURES, CFG_MID, rf_params=CFG_MID.RF_PARAMS_PAPER, do_tune=False)
display(summary_mid.tail(10))

metrics_mid = aggregate_portfolio_metrics(summary_mid, CFG_MID)
display(metrics_mid)

## Notes on methodological fixes vs the thesis

This notebook keeps the core experimental setup consistent with the thesis, but **fixes** two issues that can invalidate results:

1) **No global target filtering**: target availability is handled per training/test window, preventing sample re-weighting.  
2) **No random splits for tuning**: optional tuning uses a **date-based, purged CV** rather than shuffling.

If you want *strict* thesis replication (including tuning procedure assumptions), set `do_tune=False` and use `RF_PARAMS_PAPER`.

## 13) Turnover and simple transaction-cost sensitivity (optional)

The thesis uses an annual rebalance. Here we compute **turnover** as the fraction of names replaced year-to-year for each portfolio size, and optionally apply a **per-side cost in bps**.

In [ ]:
def compute_turnover(preds: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    # preds must include: test_year, TICKER, y_pred
    out = []
    for n in cfg.PORTFOLIO_SIZES:
        prev = None
        for year, g in preds.sort_values("test_year").groupby("test_year"):
            top = g.nlargest(n, "y_pred")[cfg.ID_COL].astype(str).tolist()
            top_set = set(top)
            if prev is None:
                out.append({"portfolio_n": n, "year": int(year), "turnover": np.nan})
            else:
                inter = len(top_set & prev)
                turnover = 1.0 - (inter / float(n))
                out.append({"portfolio_n": n, "year": int(year), "turnover": float(turnover)})
            prev = top_set
    return pd.DataFrame(out)

preds_path = os.path.join(CFG.OUT_DIR, "rf_yearly_predictions.csv")
if os.path.exists(preds_path):
    preds = pd.read_csv(preds_path)
    turnover = compute_turnover(preds, CFG)
    display(turnover.tail(20))
else:
    print("No predictions file found:", preds_path)

In [ ]:
def apply_transaction_costs(returns: pd.Series, turnover: pd.Series, cost_bps_per_side: float = 10.0) -> pd.Series:
    # Approx: annual rebalance; cost applied to traded notional.
    # If turnover is fraction of names changed, traded notional roughly equals turnover (exit+enter).
    # Multiply by 2 for both sides (sell+buy) unless you already define turnover as total traded.
    cost = (cost_bps_per_side / 10000.0) * 2.0 * turnover.fillna(0.0)
    return returns - cost

# Example sensitivity (requires turnover computed above)
if 'turnover' in locals() and not turnover.empty:
    ok = summary[summary["status"] == "OK"].copy()
    merged = ok.merge(turnover, left_on="year", right_on="year", how="left")
    for n in CFG.PORTFOLIO_SIZES:
        r = merged.loc[merged["portfolio_n"] == n, f"ret_top_{n}"].astype(float)
        t = merged.loc[merged["portfolio_n"] == n, "turnover"].astype(float)
        r_net = apply_transaction_costs(r, t, cost_bps_per_side=10.0)
        print(f"Top-{n}: mean gross={r.mean():.4f}, mean net@10bps/side={r_net.mean():.4f}")

## 14) Baselines and diagnostics (optional)

Because the end use-case is **ranking** and **portfolio selection**, MSE alone is not sufficient.

Below are simple baselines to compare against:
- **Market average**: mean 1-year return across all stocks in the investment universe on the investment date.
- **Momentum proxy** (if a momentum feature exists): rank by that feature instead of RF prediction.

You can extend this section with true benchmark index returns if you have an external series.

In [ ]:
def market_average_return_on_date(df: pd.DataFrame, date: pd.Timestamp, cfg: Config) -> float:
    x = df[df[cfg.DATE_COL] == date][cfg.TARGET_COL].dropna()
    return float(x.mean()) if len(x) else float("nan")

# Compute market-average baseline for the same investment dates used in summary
baseline = []
for _, row in summary[summary["status"]=="OK"].iterrows():
    d = pd.to_datetime(row["investment_date"])
    baseline.append({"year": int(row["year"]), "market_avg_ret": market_average_return_on_date(df, d, CFG)})

baseline = pd.DataFrame(baseline)
display(baseline.tail(10))

In [ ]:
# Example: compare RF top-N return vs market average baseline (not S&P 500)
if not baseline.empty:
    ok = summary[summary["status"]=="OK"].merge(baseline, on="year", how="left")
    for n in CFG.PORTFOLIO_SIZES:
        er = ok[f"ret_top_{n}"].astype(float) - ok["market_avg_ret"].astype(float)
        sir = er.mean() / (er.std(ddof=1) if len(er)>1 else np.nan)
        # One-tailed Welch t-test: H1 mean(RF) > mean(market_avg)
        p = ttest_ind(ok[f"ret_top_{n}"].astype(float), ok["market_avg_ret"].astype(float), equal_var=False, alternative="greater").pvalue
        print(f"Top-{n}: mean_excess_vs_market_avg={er.mean():.4f}, SIR={sir:.3f}, one-tailed Welch p={p:.4f}")

## 15) Overfitting check for tuning (train vs validation MSE)

If you enable tuning, it’s useful to compare train vs validation error to spot overfitting.

In [ ]:
def cv_train_val_mse_curve(df_train_window: pd.DataFrame, features: List[str], cfg: Config, param_list: List[Dict], sample_frac: float = 0.10):
    df_samp = df_train_window.sample(frac=min(sample_frac, 1.0), random_state=42)
    X, y, _, _, _ = prep_Xy(df_samp, df_samp, features, cfg)

    splits = purged_date_splits(df_samp.dropna(subset=[cfg.TARGET_COL]), cfg, n_splits=5)
    if not splits:
        return pd.DataFrame()

    idx_to_pos = {idx: pos for pos, idx in enumerate(X.index.to_numpy())}

    rows=[]
    for p in param_list:
        model = RandomForestRegressor(**{**cfg.RF_PARAMS_PAPER, **p})
        tr_mses=[]
        va_mses=[]
        for tr_idx, va_idx in splits:
            tr_pos=[idx_to_pos[i] for i in tr_idx if i in idx_to_pos]
            va_pos=[idx_to_pos[i] for i in va_idx if i in idx_to_pos]
            if not tr_pos or not va_pos:
                continue
            model.fit(X.iloc[tr_pos], y.iloc[tr_pos])
            pred_tr=model.predict(X.iloc[tr_pos])
            pred_va=model.predict(X.iloc[va_pos])
            tr_mses.append(mean_squared_error(y.iloc[tr_pos], pred_tr))
            va_mses.append(mean_squared_error(y.iloc[va_pos], pred_va))
        if tr_mses and va_mses:
            rows.append({**p, "train_mse": float(np.mean(tr_mses)), "val_mse": float(np.mean(va_mses))})
    return pd.DataFrame(rows)

# Example curve varying n_estimators, keeping max_depth fixed
# (Uncomment to run if needed)
# curve_params = [{"n_estimators": n, "max_depth": 13} for n in range(50, 501, 50)]
# curve = cv_train_val_mse_curve(df[df[CFG.DATE_COL] <= year_end(FIRST_Y-1)], ALL_FEATURES, CFG, curve_params, sample_frac=0.10)
# display(curve)
# if not curve.empty:
#     plt.figure()
#     plt.plot(curve["n_estimators"], curve["train_mse"])
#     plt.plot(curve["n_estimators"], curve["val_mse"])
#     plt.xlabel("n_estimators")
#     plt.ylabel("MSE")
#     plt.title("Train vs validation MSE (purged date CV)")
#     plt.show()